In [ ]:
# """
# Análisis COMPLETO del Universo - Tickers ACTIVOS + INACTIVOS
# Notebook: 01_DayBook/fase_01/A_Universo/analysis.ipynb
# """
# import polars as pl
# from pathlib import Path
# import os

# # =============================================================================
# # CONFIGURAR DIRECTORIO DE TRABAJO
# # =============================================================================
# project_root = Path(r"D:\04_TRADING_SMALLCAPS")
# os.chdir(project_root)
# print(f"📂 Directorio de trabajo: {os.getcwd()}")
# print()

# # =============================================================================
# # CARGAR DATOS - SNAPSHOT ACTIVOS
# # =============================================================================
# snapshot_path = project_root / "raw/polygon/reference/tickers_snapshot/snapshot_date=2025-10-19/tickers.parquet"
# print(f"📥 Cargando datos desde: {snapshot_path}")
# print(f"   Archivo existe: {snapshot_path.exists()}")
# print()

# df = pl.read_parquet(snapshot_path)

# print("=" * 100)
# print("ANÁLISIS UNIVERSO COMPLETO - TICKERS ACTIVOS + INACTIVOS")
# print("=" * 100)
# print()

# # =============================================================================
# # INFORMACIÓN GENERAL
# # =============================================================================
# print(f"TOTAL TICKERS (todos): {len(df):,}")
# print(f"TOTAL COLUMNAS: {len(df.columns)}")
# print()

# # Separar activos e inactivos
# df_active = df.filter(pl.col("active") == True)
# df_inactive = df.filter(pl.col("active") == False)

# print(f"✅ TICKERS ACTIVOS:   {len(df_active):>6,} ({len(df_active)/len(df)*100:>5.1f}%)")
# print(f"❌ TICKERS INACTIVOS: {len(df_inactive):>6,} ({len(df_inactive)/len(df)*100:>5.1f}%)")
# print()

# # =============================================================================
# # COLUMNAS Y TIPOS DE DATOS
# # =============================================================================
# print("=" * 100)
# print("COLUMNAS Y TIPOS DE DATOS")
# print("=" * 100)
# for col in df.columns:
#     dtype = df[col].dtype
#     null_count = df[col].null_count()
#     null_pct = (null_count / len(df)) * 100
#     print(f"{col:30s} | {str(dtype):20s} | Nulls: {null_count:6,} ({null_pct:5.1f}%)")
# print()

# # =============================================================================
# # HEAD(10) - PRIMERAS 10 ACCIONES (TODOS)
# # =============================================================================
# print("=" * 100)
# print("HEAD(10) - PRIMERAS 10 ACCIONES (ACTIVOS + INACTIVOS)")
# print("=" * 100)
# cols_display = ["ticker", "name", "market", "primary_exchange", "type", "active", "cik"]
# display(df.select(cols_display).head(10))
# print()

# # =============================================================================
# # HEAD(10) - INACTIVOS
# # =============================================================================
# if len(df_inactive) > 0:
#     print("=" * 100)
#     print(f"HEAD(10) - PRIMERAS 10 ACCIONES INACTIVAS (Total: {len(df_inactive):,})")
#     print("=" * 100)
#     display(df_inactive.select(cols_display).head(10))
#     print()

# # =============================================================================
# # TAIL(10) - ÚLTIMAS 10 ACCIONES
# # =============================================================================
# print("=" * 100)
# print("TAIL(10) - ÚLTIMAS 10 ACCIONES")
# print("=" * 100)
# display(df.select(cols_display).tail(10))
# print()

# # =============================================================================
# # COMPARACIÓN: ACTIVOS VS INACTIVOS
# # =============================================================================
# print("=" * 100)
# print("COMPARACIÓN: ACTIVOS VS INACTIVOS")
# print("=" * 100)

# # Por tipo de activo
# print("\n DISTRIBUCIÓN POR TIPO DE ACTIVO (type):")
# print("\n   ACTIVOS:")
# type_dist_active = (
#     df_active.group_by("type")
#     .agg(pl.len().alias("count"))
#     .with_columns((pl.col("count") / len(df_active) * 100).alias("percentage"))
#     .sort("count", descending=True)
#     .head(10)
# )
# display(type_dist_active)

# if len(df_inactive) > 0:
#     print("\n   INACTIVOS:")
#     type_dist_inactive = (
#         df_inactive.group_by("type")
#         .agg(pl.len().alias("count"))
#         .with_columns((pl.col("count") / len(df_inactive) * 100).alias("percentage"))
#         .sort("count", descending=True)
#         .head(10)
#     )
#     display(type_dist_inactive)

# # Por exchange
# print("\n🏛️ DISTRIBUCIÓN POR PRIMARY EXCHANGE:")
# print("\n   ACTIVOS:")
# exchange_dist_active = (
#     df_active.group_by("primary_exchange")
#     .agg(pl.len().alias("count"))
#     .with_columns((pl.col("count") / len(df_active) * 100).alias("percentage"))
#     .sort("count", descending=True)
# )
# display(exchange_dist_active)

# if len(df_inactive) > 0:
#     print("\n   INACTIVOS:")
#     exchange_dist_inactive = (
#         df_inactive.group_by("primary_exchange")
#         .agg(pl.len().alias("count"))
#         .with_columns((pl.col("count") / len(df_inactive) * 100).alias("percentage"))
#         .sort("count", descending=True)
#         .head(10)
#     )
#     display(exchange_dist_inactive)

# # =============================================================================
# # ESTADÍSTICAS DE CIK
# # =============================================================================
# print("\n📋 ESTADÍSTICAS DE CIK (SEC Identifier):")
# print("\n   ACTIVOS:")
# cik_active = df_active.filter(pl.col("cik").is_not_null())
# print(f"      ✅ Con CIK: {len(cik_active):,} ({len(cik_active)/len(df_active)*100:.1f}%)")
# print(f"      ❌ Sin CIK: {df_active['cik'].null_count():,} ({df_active['cik'].null_count()/len(df_active)*100:.1f}%)")

# if len(df_inactive) > 0:
#     print("\n   INACTIVOS:")
#     cik_inactive = df_inactive.filter(pl.col("cik").is_not_null())
#     print(f"      ✅ Con CIK: {len(cik_inactive):,} ({len(cik_inactive)/len(df_inactive)*100:.1f}%)")
#     print(f"      ❌ Sin CIK: {df_inactive['cik'].null_count():,} ({df_inactive['cik'].null_count()/len(df_inactive)*100:.1f}%)")
# print()

# # =============================================================================
# # ESTADÍSTICAS DE FIGI
# # =============================================================================
# print("📋 ESTADÍSTICAS DE FIGI (Bloomberg Identifier):")
# print("\n   ACTIVOS:")
# figi_active = df_active.filter(pl.col("composite_figi").is_not_null())
# print(f"      ✅ Con FIGI: {len(figi_active):,} ({len(figi_active)/len(df_active)*100:.1f}%)")
# print(f"      ❌ Sin FIGI: {df_active['composite_figi'].null_count():,} ({df_active['composite_figi'].null_count()/len(df_active)*100:.1f}%)")

# if len(df_inactive) > 0:
#     print("\n   INACTIVOS:")
#     figi_inactive = df_inactive.filter(pl.col("composite_figi").is_not_null())
#     print(f"      ✅ Con FIGI: {len(figi_inactive):,} ({len(figi_inactive)/len(df_inactive)*100:.1f}%)")
#     print(f"      ❌ Sin FIGI: {df_inactive['composite_figi'].null_count():,} ({df_inactive['composite_figi'].null_count()/len(df_inactive)*100:.1f}%)")
# print()

# # =============================================================================
# # RESUMEN EJECUTIVO - ACTIVOS
# # =============================================================================
# print("=" * 100)
# print("RESUMEN EJECUTIVO - TICKERS ACTIVOS")
# print("=" * 100)
# print(f"""
#  UNIVERSO ACTIVO: {len(df_active):,} tickers

#  COMPOSICIÓN:
#    • Common Stocks (CS):    {len(df_active.filter(pl.col('type') == 'CS')):>5,} ({len(df_active.filter(pl.col('type') == 'CS'))/len(df_active)*100:>5.1f}%)
#    • ETFs:                  {len(df_active.filter(pl.col('type') == 'ETF')):>5,} ({len(df_active.filter(pl.col('type') == 'ETF'))/len(df_active)*100:>5.1f}%)
#    • Preferred (PFD):       {len(df_active.filter(pl.col('type') == 'PFD')):>5,} ({len(df_active.filter(pl.col('type') == 'PFD'))/len(df_active)*100:>5.1f}%)
#    • Warrants:              {len(df_active.filter(pl.col('type') == 'WARRANT')):>5,} ({len(df_active.filter(pl.col('type') == 'WARRANT'))/len(df_active)*100:>5.1f}%)
#    • ADRs:                  {len(df_active.filter(pl.col('type') == 'ADRC')):>5,} ({len(df_active.filter(pl.col('type') == 'ADRC'))/len(df_active)*100:>5.1f}%)
#    • Funds:                 {len(df_active.filter(pl.col('type') == 'FUND')):>5,} ({len(df_active.filter(pl.col('type') == 'FUND'))/len(df_active)*100:>5.1f}%)

#  EXCHANGES:
#    • Nasdaq (XNAS):         {len(df_active.filter(pl.col('primary_exchange') == 'XNAS')):>5,} ({len(df_active.filter(pl.col('primary_exchange') == 'XNAS'))/len(df_active)*100:>5.1f}%)
#    • NYSE (XNYS):           {len(df_active.filter(pl.col('primary_exchange') == 'XNYS')):>5,} ({len(df_active.filter(pl.col('primary_exchange') == 'XNYS'))/len(df_active)*100:>5.1f}%)
#    • NYSE Arca (ARCX):      {len(df_active.filter(pl.col('primary_exchange') == 'ARCX')):>5,} ({len(df_active.filter(pl.col('primary_exchange') == 'ARCX'))/len(df_active)*100:>5.1f}%)
#    • BATS:                  {len(df_active.filter(pl.col('primary_exchange') == 'BATS')):>5,} ({len(df_active.filter(pl.col('primary_exchange') == 'BATS'))/len(df_active)*100:>5.1f}%)
#    • NYSE American (XASE):  {len(df_active.filter(pl.col('primary_exchange') == 'XASE')):>5,} ({len(df_active.filter(pl.col('primary_exchange') == 'XASE'))/len(df_active)*100:>5.1f}%)

#  CALIDAD DE DATOS:
#    • {len(cik_active)/len(df_active)*100:.1f}% con CIK (SEC identifier)
#    • {len(figi_active)/len(df_active)*100:.1f}% con FIGI (Bloomberg identifier)
# """)

# # =============================================================================
# # RESUMEN EJECUTIVO - INACTIVOS
# # =============================================================================
# if len(df_inactive) > 0:
#     print("=" * 100)
#     print("RESUMEN EJECUTIVO - TICKERS INACTIVOS")
#     print("=" * 100)
    
#     cik_inactive_calc = df_inactive.filter(pl.col("cik").is_not_null())
#     figi_inactive_calc = df_inactive.filter(pl.col("composite_figi").is_not_null())
    
#     print(f"""
#  UNIVERSO INACTIVO: {len(df_inactive):,} tickers (delisted/merged/inactive)

#  COMPOSICIÓN:
#    • Common Stocks (CS):    {len(df_inactive.filter(pl.col('type') == 'CS')):>5,} ({len(df_inactive.filter(pl.col('type') == 'CS'))/len(df_inactive)*100:>5.1f}%)
#    • ETFs:                  {len(df_inactive.filter(pl.col('type') == 'ETF')):>5,} ({len(df_inactive.filter(pl.col('type') == 'ETF'))/len(df_inactive)*100:>5.1f}%)
#    • Preferred (PFD):       {len(df_inactive.filter(pl.col('type') == 'PFD')):>5,} ({len(df_inactive.filter(pl.col('type') == 'PFD'))/len(df_inactive)*100:>5.1f}%)
#    • Warrants:              {len(df_inactive.filter(pl.col('type') == 'WARRANT')):>5,} ({len(df_inactive.filter(pl.col('type') == 'WARRANT'))/len(df_inactive)*100:>5.1f}%)

#  CALIDAD DE DATOS:
#    • {len(cik_inactive_calc)/len(df_inactive)*100:.1f}% con CIK (SEC identifier)
#    • {len(figi_inactive_calc)/len(df_inactive)*100:.1f}% con FIGI (Bloomberg identifier)
# """)

# print("=" * 100)
# print("ANÁLISIS COMPLETADO")
# print("=" * 100)

In [ ]:
"""
Análisis Completo: Tickers Activos + Inactivos
Snapshot: 2025-10-24
"""
import polars as pl
from pathlib import Path
import os

# Configurar
project_root = Path(r"D:\04_TRADING_SMALLCAPS")
os.chdir(project_root)

# =============================================================================
# CARGAR RESUMEN CSV
# =============================================================================
csv_path = "temp_active_counts_complete.csv"
df_summary = pl.read_csv(csv_path)

print("="*100)
print("RESUMEN GENERAL")
print("="*100)
display(df_summary)
print()

# =============================================================================
# CARGAR SNAPSHOT COMPLETO
# =============================================================================
snapshot_path = "raw/polygon/reference/tickers_snapshot/snapshot_date=2025-10-24/tickers_all.parquet"
df_all = pl.read_parquet(snapshot_path)

print(f"TOTAL TICKERS: {len(df_all):,}")
print()

# Separar activos e inactivos
df_active = df_all.filter(pl.col("active") == True)
df_inactive = df_all.filter(pl.col("active") == False)

print(f"ACTIVOS:   {len(df_active):>8,} ({len(df_active)/len(df_all)*100:>5.1f}%)")
print(f"INACTIVOS: {len(df_inactive):>8,} ({len(df_inactive)/len(df_all)*100:>5.1f}%)")
print()

# =============================================================================
# HEAD(10) - ACTIVOS
# =============================================================================
print("="*100)
print("HEAD(10) - TICKERS ACTIVOS")
print("="*100)
cols = ["ticker", "name", "market", "primary_exchange", "type", "active", "cik"]
display(df_active.select(cols).head(10))

# =============================================================================
# HEAD(10) - INACTIVOS
# =============================================================================
print()
print("="*100)
print(f"HEAD(10) - TICKERS INACTIVOS (Total: {len(df_inactive):,})")
print("="*100)
display(df_inactive.select(cols).head(10))

# =============================================================================
# TAIL(10) - INACTIVOS  
# =============================================================================
print()
print("="*100)
print("TAIL(10) - TICKERS INACTIVOS")
print("="*100)
display(df_inactive.select(cols).tail(10))

# =============================================================================
# COMPARACIÓN POR TIPO
# =============================================================================
print()
print("="*100)
print("COMPARACIÓN: DISTRIBUCIÓN POR TIPO DE ACTIVO")
print("="*100)

print("\n📊 ACTIVOS:")
type_active = (
    df_active.group_by("type")
    .agg(pl.len().alias("count"))
    .with_columns((pl.col("count") / len(df_active) * 100).alias("percentage"))
    .sort("count", descending=True)
    .head(10)
)
display(type_active)

print("\n❌ INACTIVOS:")
type_inactive = (
    df_inactive.group_by("type")
    .agg(pl.len().alias("count"))
    .with_columns((pl.col("count") / len(df_inactive) * 100).alias("percentage"))
    .sort("count", descending=True)
    .head(10)
)
display(type_inactive)

# =============================================================================
# COMPARACIÓN POR EXCHANGE
# =============================================================================
print()
print("="*100)
print("COMPARACIÓN: DISTRIBUCIÓN POR EXCHANGE")
print("="*100)

print("\n📊 ACTIVOS:")
exchange_active = (
    df_active.group_by("primary_exchange")
    .agg(pl.len().alias("count"))
    .with_columns((pl.col("count") / len(df_active) * 100).alias("percentage"))
    .sort("count", descending=True)
)
display(exchange_active)

print("\n❌ INACTIVOS:")
exchange_inactive = (
    df_inactive.group_by("primary_exchange")
    .agg(pl.len().alias("count"))
    .with_columns((pl.col("count") / len(df_inactive) * 100).alias("percentage"))
    .sort("count", descending=True)
    .head(10)
)
display(exchange_inactive)

# =============================================================================
# RESUMEN EJECUTIVO
# =============================================================================
print()
print("="*100)
print("RESUMEN EJECUTIVO")
print("="*100)
print(f"""
📊 UNIVERSO TOTAL: {len(df_all):,} tickers

ACTIVOS ({len(df_active):,}):
   • Common Stocks (CS):  {len(df_active.filter(pl.col('type') == 'CS')):>6,}
   • ETFs:                {len(df_active.filter(pl.col('type') == 'ETF')):>6,}
   • Nasdaq (XNAS):       {len(df_active.filter(pl.col('primary_exchange') == 'XNAS')):>6,}
   • NYSE (XNYS):         {len(df_active.filter(pl.col('primary_exchange') == 'XNYS')):>6,}

INACTIVOS ({len(df_inactive):,}):
   • Common Stocks (CS):  {len(df_inactive.filter(pl.col('type') == 'CS')):>6,}
   • Warrants:            {len(df_inactive.filter(pl.col('type') == 'WARRANT')):>6,}
   • ETFs:                {len(df_inactive.filter(pl.col('type') == 'ETF')):>6,}
   • Delisted/Merged:     {len(df_inactive):,} (100% inactivos)
""")

print("="*100)
print("ANÁLISIS COMPLETADO")
print("="*100)

In [1]:
"""
Análisis: Qué nos queda al aplicar filtro CS (Common Stock)
Muestra el antes y después del filtrado
"""
import polars as pl
from pathlib import Path
import os

# Configurar
project_root = Path(r"D:\04_TRADING_SMALLCAPS")
os.chdir(project_root)

# Cargar datos
df = pl.read_parquet("raw/polygon/reference/tickers_snapshot/snapshot_date=2025-10-24/tickers_all.parquet")

print("=" * 100)
print("QUE NOS QUEDA AL APLICAR FILTRO CS (Common Stock)")
print("=" * 100)

print(f"\n[ANTES DEL FILTRO] UNIVERSO TOTAL: {len(df):,} tickers (TODOS los tipos)")
print()

# Separar activos e inactivos ANTES del filtro
df_active_all = df.filter(pl.col("active") == True)
df_inactive_all = df.filter(pl.col("active") == False)

print("COMPOSICION ANTES DEL FILTRO:")
print(f"  ACTIVOS (TODOS los tipos):   {len(df_active_all):>7,}")
print(f"    ├─ CS (Common Stock):       {len(df_active_all.filter(pl.col('type') == 'CS')):>7,}")
print(f"    ├─ ETF:                     {len(df_active_all.filter(pl.col('type') == 'ETF')):>7,}")
print(f"    ├─ WARRANT:                 {len(df_active_all.filter(pl.col('type') == 'WARRANT')):>7,}")
print(f"    ├─ PFD (Preferred):         {len(df_active_all.filter(pl.col('type') == 'PFD')):>7,}")
print(f"    └─ Otros:                   {len(df_active_all) - len(df_active_all.filter(pl.col('type').is_in(['CS', 'ETF', 'WARRANT', 'PFD']))):>7,}")
print()
print(f"  INACTIVOS (TODOS los tipos): {len(df_inactive_all):>7,}")
print(f"    ├─ CS (Common Stock):       {len(df_inactive_all.filter(pl.col('type') == 'CS')):>7,}")
print(f"    ├─ WARRANT:                 {len(df_inactive_all.filter(pl.col('type') == 'WARRANT')):>7,}")
print(f"    ├─ ETF:                     {len(df_inactive_all.filter(pl.col('type') == 'ETF')):>7,}")
print(f"    └─ Otros:                   {len(df_inactive_all) - len(df_inactive_all.filter(pl.col('type').is_in(['CS', 'WARRANT', 'ETF']))):>7,}")

print("\n" + "=" * 100)
print("APLICANDO FILTRO: type = 'CS' (Common Stock)")
print("=" * 100)

# Filtrar solo CS
df_cs = df.filter(pl.col("type") == "CS")
df_cs_active = df_cs.filter(pl.col("active") == True)
df_cs_inactive = df_cs.filter(pl.col("active") == False)

print(f"\n[DESPUES DEL FILTRO] QUEDAN: {len(df_cs):,} tickers (SOLO Common Stock)")
print()
print("COMPOSICION DESPUES DEL FILTRO:")
print(f"  ACTIVOS CS:                   {len(df_cs_active):>7,} ({len(df_cs_active)/len(df_cs)*100:>5.1f}% de CS)")
print(f"  INACTIVOS CS:                 {len(df_cs_inactive):>7,} ({len(df_cs_inactive)/len(df_cs)*100:>5.1f}% de CS)")

print("\n" + "=" * 100)
print("APLICANDO FILTRO ADICIONAL: primary_exchange IN ('XNAS', 'XNYS')")
print("=" * 100)

# Filtrar CS + XNAS/XNYS
df_cs_exchanges = df_cs.filter(pl.col("primary_exchange").is_in(["XNAS", "XNYS"]))
df_cs_exchanges_active = df_cs_exchanges.filter(pl.col("active") == True)
df_cs_exchanges_inactive = df_cs_exchanges.filter(pl.col("active") == False)

# Por exchange
df_cs_xnas = df_cs_exchanges.filter(pl.col("primary_exchange") == "XNAS")
df_cs_xnys = df_cs_exchanges.filter(pl.col("primary_exchange") == "XNYS")

print(f"\n[DESPUES DE AMBOS FILTROS] QUEDAN: {len(df_cs_exchanges):,} tickers")
print("(Solo Common Stock en NASDAQ o NYSE)")
print()
print("COMPOSICION FINAL:")
print(f"  TOTAL CS en XNAS/XNYS:        {len(df_cs_exchanges):>7,}")
print(f"    ├─ ACTIVOS:                 {len(df_cs_exchanges_active):>7,} ({len(df_cs_exchanges_active)/len(df_cs_exchanges)*100:>5.1f}%)")
print(f"    └─ INACTIVOS:               {len(df_cs_exchanges_inactive):>7,} ({len(df_cs_exchanges_inactive)/len(df_cs_exchanges)*100:>5.1f}%)")
print()
print("  POR EXCHANGE:")
print(f"    ├─ NASDAQ (XNAS):           {len(df_cs_xnas):>7,} ({len(df_cs_xnas)/len(df_cs_exchanges)*100:>5.1f}%)")
print(f"    │  ├─ Activos:              {len(df_cs_xnas.filter(pl.col('active') == True)):>7,}")
print(f"    │  └─ Inactivos:            {len(df_cs_xnas.filter(pl.col('active') == False)):>7,}")
print(f"    └─ NYSE (XNYS):             {len(df_cs_xnys):>7,} ({len(df_cs_xnys)/len(df_cs_exchanges)*100:>5.1f}%)")
print(f"       ├─ Activos:              {len(df_cs_xnys.filter(pl.col('active') == True)):>7,}")
print(f"       └─ Inactivos:            {len(df_cs_xnys.filter(pl.col('active') == False)):>7,}")

print("\n" + "=" * 100)
print("RESUMEN: QUE SE ELIMINA CON EL FILTRO CS")
print("=" * 100)

eliminados = len(df) - len(df_cs)
print(f"\nELIMINADOS: {eliminados:,} tickers (que NO son Common Stock)")
print(f"  ├─ ETFs:                      {len(df.filter(pl.col('type') == 'ETF')):>7,}")
print(f"  ├─ Warrants:                  {len(df.filter(pl.col('type') == 'WARRANT')):>7,}")
print(f"  ├─ Preferred (PFD):           {len(df.filter(pl.col('type') == 'PFD')):>7,}")
print(f"  ├─ Funds:                     {len(df.filter(pl.col('type') == 'FUND')):>7,}")
print(f"  ├─ Units:                     {len(df.filter(pl.col('type') == 'UNIT')):>7,}")
print(f"  ├─ ADRs:                      {len(df.filter(pl.col('type') == 'ADRC')):>7,}")
print(f"  ├─ Rights:                    {len(df.filter(pl.col('type') == 'RIGHT')):>7,}")
print(f"  ├─ Indices:                   {len(df.filter(pl.col('type') == 'INDEX')):>7,}")
print(f"  ├─ Sin tipo (None):           {len(df.filter(pl.col('type').is_null())):>7,}")

otros_count = eliminados - sum([
    len(df.filter(pl.col('type') == t)) 
    for t in ['ETF', 'WARRANT', 'PFD', 'FUND', 'UNIT', 'ADRC', 'RIGHT', 'INDEX']
]) - len(df.filter(pl.col('type').is_null()))
print(f"  └─ Otros:                     {otros_count:>7,}")

print(f"\nMANTENIDOS: {len(df_cs):,} Common Stocks (CS)")
print(f"  De los cuales {len(df_cs_exchanges):,} estan en NASDAQ o NYSE")

print("\n" + "=" * 100)
print("CONCLUSION: Este es el universo base (10,599 CS) para descargar ticker details")
print("Proximo paso: Filtrar por market_cap < $2B")
print("=" * 100)

QUE NOS QUEDA AL APLICAR FILTRO CS (Common Stock)

[ANTES DEL FILTRO] UNIVERSO TOTAL: 34,380 tickers (TODOS los tipos)

COMPOSICION ANTES DEL FILTRO:
  ACTIVOS (TODOS los tipos):    11,853
    ├─ CS (Common Stock):         5,229
    ├─ ETF:                       4,365
    ├─ WARRANT:                     417
    ├─ PFD (Preferred):             441
    └─ Otros:                     1,401

  INACTIVOS (TODOS los tipos):  22,527
    ├─ CS (Common Stock):         6,242
    ├─ WARRANT:                   1,724
    ├─ ETF:                       1,363
    └─ Otros:                    13,198

APLICANDO FILTRO: type = 'CS' (Common Stock)

[DESPUES DEL FILTRO] QUEDAN: 11,471 tickers (SOLO Common Stock)

COMPOSICION DESPUES DEL FILTRO:
  ACTIVOS CS:                     5,229 ( 45.6% de CS)
  INACTIVOS CS:                   6,242 ( 54.4% de CS)

APLICANDO FILTRO ADICIONAL: primary_exchange IN ('XNAS', 'XNYS')

[DESPUES DE AMBOS FILTROS] QUEDAN: 10,599 tickers
(Solo Common Stock en NASDAQ o NYSE)

CO

In [ ]:
import pandas as pd
from pathlib import Path

# Cargar con pandas (más lento pero más compatible)
project_root = Path(r"D:\04_TRADING_SMALLCAPS")
snapshot_path = project_root / "raw/polygon/reference/tickers_snapshot/snapshot_date=2025-10-24/tickers_all.parquet"

print("Cargando datos...")
df = pd.read_parquet(snapshot_path)
print(f"✓ Datos cargados: {len(df):,} tickers\n")

# Distribución por TYPE
type_dist = df['type'].fillna('None').value_counts().reset_index()
type_dist.columns = ['Tipo', 'Cantidad']
type_dist['% Total'] = (type_dist['Cantidad'] / len(df) * 100).round(1)

# Descripciones
type_descriptions = {
    'CS': 'Common Stock (acciones comunes)',
    'ETF': 'Exchange Traded Funds',
    'PFD': 'Preferred Stock (acciones preferentes)',
    'SP': 'Security Pass-Through',
    'WARRANT': 'Warrants',
    'UNIT': 'Units',
    'ADRC': 'ADR Common',
    'FUND': 'Mutual Funds',
    'RIGHT': 'Rights',
    'INDEX': 'Indices',
    'ETN': 'Exchange Traded Notes',
    'ETS': 'Exchange Traded Shares',
    'ETV': 'Exchange Traded Vehicles',
    'ADRP': 'ADR Preferred',
    'ADRR': 'ADR Rights',
    'None': 'Sin clasificar'
}

type_dist['Descripción'] = type_dist['Tipo'].map(type_descriptions)
display(type_dist)

# Información de columnas
print(f"\nCOLUMNAS ({len(df.columns)}):")
print(df.dtypes)
print(f"\nNULLS:")
print(df.isnull().sum())

**CODIGO PARA LANZAR LA DESCARGA DE TIKERS DE TODOS LOS CS activos e inactovos**

```sh
cd D:/04_TRADING_SMALLCAPS && POLYGON_API_KEY= 
python scripts/fase_A_universo/filter_and_enrich_universe.py 2>&1 &
```

In [2]:
"""
Verificación del Filtrado CS + Market Cap < $2B
Análisis completo del universo final de small caps
"""
import polars as pl
from pathlib import Path
import os

# Configurar
project_root = Path(r"D:\04_TRADING_SMALLCAPS")
os.chdir(project_root)

print("=" * 100)
print("VERIFICACIÓN DEL PROCESO DE FILTRADO")
print("=" * 100)

# =============================================================================
# PASO 1: Verificar snapshot original
# =============================================================================
print("\n[PASO 1] SNAPSHOT ORIGINAL")
print("-" * 100)

snapshot_path = "raw/polygon/reference/tickers_snapshot/snapshot_date=2025-10-24/cs_xnas_xnys_hybrid_2025-10-24.parquet"
df_original = pl.read_parquet(snapshot_path)

print(f"Total tickers originales: {len(df_original):,}")
print(f"  - Activos:   {len(df_original.filter(pl.col('active') == True)):,}")
print(f"  - Inactivos: {len(df_original.filter(pl.col('active') == False)):,}")

# =============================================================================
# PASO 2: Verificar filtro CS + Nasdaq (XNAS) NYSE (XNYS)
# =============================================================================
print("\n[PASO 2] FILTRO CS + XNAS/XNYS")
print("-" * 100)

cs_filtered_path = "processed/universe/cs_all_xnas_xnys.parquet"
df_cs_filtered = pl.read_parquet(cs_filtered_path)

print(f"CS en XNAS/XNYS: {len(df_cs_filtered):,}")
print(f"  - Activos:   {len(df_cs_filtered.filter(pl.col('active') == True)):,}")
print(f"  - Inactivos: {len(df_cs_filtered.filter(pl.col('active') == False)):,}")
print()
print("Distribución por exchange:")
for exchange in ["XNAS", "XNYS"]:
    total = len(df_cs_filtered.filter(pl.col("primary_exchange") == exchange))
    active = len(df_cs_filtered.filter((pl.col("primary_exchange") == exchange) & (pl.col("active") == True)))
    inactive = total - active
    print(f"  {exchange}: {total:>6,} total ({active:>5,} activos, {inactive:>5,} inactivos)")

# =============================================================================
# PASO 3: Verificar ticker details descargados
# =============================================================================
print("\n[PASO 3] TICKER DETAILS DESCARGADOS")
print("-" * 100)

details_path = "raw/polygon/reference/ticker_details/ticker_details_2025-10-24.parquet"
df_details = pl.read_parquet(details_path)

print(f"Total details descargados: {len(df_details):,}")
print(f"Con market_cap data:       {len(df_details.filter(pl.col('market_cap').is_not_null())):,}")
print(f"Sin market_cap (null):     {len(df_details.filter(pl.col('market_cap').is_null())):,}")
print(f"Con error (404):           {len(df_details.filter(pl.col('ticker').str.contains('error', literal=False) if 'error' in df_details.columns else pl.lit(False))):,}")

# =============================================================================
# PASO 4: Verificar universo final < $2B
# =============================================================================
print("\n[PASO 4] UNIVERSO FINAL < $2B")
print("-" * 100)

final_path = "processed/universe/cs_xnas_xnys_under2b_all_2025-10-24.parquet"
df_final = pl.read_parquet(final_path)

print(f"Small caps < $2B: {len(df_final):,}")

# Verificar si existe columna 'active'
if 'active' in df_final.columns:
    final_active = len(df_final.filter(pl.col("active") == True))
    final_inactive = len(df_final) - final_active
    print(f"  - Activos:   {final_active:>6,} ({final_active/len(df_final)*100:>5.1f}%)")
    print(f"  - Inactivos: {final_inactive:>6,} ({final_inactive/len(df_final)*100:>5.1f}%)")
else:
    print("  [Nota: columna 'active' no encontrada en el resultado final]")

# Distribución por exchange (si existe la columna)
if 'primary_exchange' in df_final.columns:
    print()
    print("Distribución por exchange:")
    exchange_dist = df_final.group_by("primary_exchange").len(name="count").sort("count", descending=True)
    for row in exchange_dist.iter_rows(named=True):
        exchange = row['primary_exchange'] if row['primary_exchange'] else 'None'
        count = row['count']
        pct = count / len(df_final) * 100
        print(f"  {exchange}: {count:>6,} ({pct:>5.1f}%)")

# =============================================================================
# PASO 5: Análisis de market_cap
# =============================================================================
print("\n[PASO 5] ANÁLISIS DE MARKET CAP")
print("-" * 100)

if 'market_cap' in df_final.columns:
    df_with_cap = df_final.filter(pl.col("market_cap").is_not_null())
    
    print(f"Tickers con market_cap: {len(df_with_cap):,}")
    print()
    print("Estadísticas de market cap:")
    stats = df_with_cap.select(pl.col("market_cap")).describe()
    print(stats)
    
    print()
    print("Rangos de market cap:")
    ranges = [
        ("Micro cap (< $300M)", 0, 300_000_000),
        ("Small cap ($300M - $2B)", 300_000_000, 2_000_000_000),
    ]
    
    for label, min_val, max_val in ranges:
        count = len(df_with_cap.filter(
            (pl.col("market_cap") >= min_val) & (pl.col("market_cap") < max_val)
        ))
        pct = count / len(df_with_cap) * 100
        print(f"  {label}: {count:>6,} ({pct:>5.1f}%)")
    
    # Verificar si hay alguno > $2B (error en filtrado)
    over_2b = len(df_with_cap.filter(pl.col("market_cap") >= 2_000_000_000))
    if over_2b > 0:
        print(f"\n⚠️  ADVERTENCIA: {over_2b} tickers con market_cap >= $2B (error en filtrado)")
else:
    print("⚠️  Columna 'market_cap' no encontrada en resultado final")

# =============================================================================
# PASO 6: Muestreo de datos
# =============================================================================
print("\n[PASO 6] MUESTREO DE DATOS FINALES")
print("-" * 100)

print("\nPrimeros 10 tickers (ordenados alfabéticamente):")
cols_display = ["ticker", "name", "market_cap", "primary_exchange", "active"]
cols_available = [col for col in cols_display if col in df_final.columns]

if 'ticker' in df_final.columns:
    display(df_final.sort("ticker").select(cols_available).head(10))
else:
    print("⚠️  Columna 'ticker' no encontrada")

print("\nÚltimos 10 tickers:")
if 'ticker' in df_final.columns:
    display(df_final.sort("ticker").select(cols_available).tail(10))

# =============================================================================
# PASO 7: Resumen ejecutivo
# =============================================================================
print("\n" + "=" * 100)
print("RESUMEN EJECUTIVO")
print("=" * 100)

print(f"""
PIPELINE COMPLETO:
  1. Snapshot original:          {len(df_original):>6,} tickers (todos los tipos)
  2. Filtro CS + XNAS/XNYS:      {len(df_cs_filtered):>6,} tickers
  3. Details descargados:        {len(df_details):>6,} tickers
  4. Con market_cap data:        {len(df_details.filter(pl.col('market_cap').is_not_null())):>6,} tickers
  5. Filtro < $2B:               {len(df_final):>6,} tickers ✅

UNIVERSO FINAL SMALL CAPS:
  - Total: {len(df_final):,} tickers
  - Common Stock (CS) solamente
  - NASDAQ (XNAS) y NYSE (XNYS) únicamente
  - Market cap < $2B
  - Incluye activos + inactivos (sin sesgo de supervivencia)

ARCHIVOS GENERADOS:
  ✅ {cs_filtered_path}
  ✅ {details_path}
  ✅ {final_path}
  ✅ processed/universe/cs_xnas_xnys_under2b_all_2025-10-24.csv
""")

print("=" * 100)
print("VERIFICACIÓN COMPLETADA")
print("=" * 100)

VERIFICACIÓN DEL PROCESO DE FILTRADO

[PASO 1] SNAPSHOT ORIGINAL
----------------------------------------------------------------------------------------------------


FileNotFoundError: El sistema no puede encontrar el archivo especificado. (os error 2): ...ference/tickers_snapshot/snapshot_date=2025-10-24/cs_xnas_xnys_hybrid_2025-10-24.parquet (set POLARS_VERBOSE=1 to see full path)

This error occurred with the following context stack:
	[1] 'parquet scan'
	[2] 'sink'


In [ ]:
"""
Verificación del Universo Híbrido (Activos < $2B + Inactivos ALL)
Análisis completo sin sesgo de supervivencia
"""
import polars as pl
from pathlib import Path
import os

# Configurar
project_root = Path(r"D:\04_TRADING_SMALLCAPS")
os.chdir(project_root)

print("=" * 100)
print("VERIFICACIÓN DEL UNIVERSO HÍBRIDO (SIN SESGO DE SUPERVIVENCIA)")
print("=" * 100)

# =============================================================================
# PASO 1: Verificar snapshot original (TODOS los tipos)
# =============================================================================
print("\n[PASO 1] SNAPSHOT ORIGINAL (TODOS LOS TIPOS)")
print("-" * 100)

snapshot_path = "raw/polygon/reference/tickers_snapshot/snapshot_date=2025-10-24/tickers_all.parquet"
df_original = pl.read_parquet(snapshot_path)

print(f"Total tickers originales: {len(df_original):,}")
print(f"  - Activos:   {len(df_original.filter(pl.col('active') == True)):,}")
print(f"  - Inactivos: {len(df_original.filter(pl.col('active') == False)):,}")

# Distribución por tipo
print("\nDistribución por tipo (top 5):")
type_dist = df_original.group_by("type").len(name="count").sort("count", descending=True)
for row in type_dist.head(5).iter_rows(named=True):
    tipo = row['type'] if row['type'] else 'None'
    count = row['count']
    pct = count / len(df_original) * 100
    print(f"  {tipo:<15} {count:>6,} ({pct:>5.1f}%)")

# =============================================================================
# PASO 2: Verificar filtro CS + XNAS/XNYS
# =============================================================================
print("\n[PASO 2] FILTRO CS + XNAS/XNYS (BASE)")
print("-" * 100)

cs_filtered_path = "processed/universe/cs_all_xnas_xnys.parquet"
df_cs_filtered = pl.read_parquet(cs_filtered_path)

print(f"CS en XNAS/XNYS: {len(df_cs_filtered):,}")
print(f"  - Activos:   {len(df_cs_filtered.filter(pl.col('active') == True)):,}")
print(f"  - Inactivos: {len(df_cs_filtered.filter(pl.col('active') == False)):,}")
print()
print("Distribución por exchange:")
for exchange in ["XNAS", "XNYS"]:
    total = len(df_cs_filtered.filter(pl.col("primary_exchange") == exchange))
    active = len(df_cs_filtered.filter((pl.col("primary_exchange") == exchange) & (pl.col("active") == True)))
    inactive = total - active
    print(f"  {exchange}: {total:>6,} total ({active:>5,} activos, {inactive:>5,} inactivos)")

# =============================================================================
# PASO 3: Verificar ticker details descargados
# =============================================================================
print("\n[PASO 3] TICKER DETAILS DESCARGADOS")
print("-" * 100)

details_path = "raw/polygon/reference/ticker_details/ticker_details_2025-10-24.parquet"
df_details = pl.read_parquet(details_path)

print(f"Total details descargados: {len(df_details):,}")
print(f"Con market_cap data:       {len(df_details.filter(pl.col('market_cap').is_not_null())):,}")
print(f"Sin market_cap (null):     {len(df_details.filter(pl.col('market_cap').is_null())):,}")

# Análisis de activos/inactivos en details
if 'active' in df_details.columns:
    active_details = len(df_details.filter(pl.col('active') == True))
    inactive_details = len(df_details.filter(pl.col('active') == False))
    print(f"\nBreakdown por active:")
    print(f"  - Activos:   {active_details:,}")
    print(f"  - Inactivos: {inactive_details:,}")

# =============================================================================
# PASO 4: Verificar universo HÍBRIDO (activos < $2B + inactivos ALL)
# =============================================================================
print("\n[PASO 4] UNIVERSO HÍBRIDO FINAL")
print("-" * 100)

hybrid_path = "processed/universe/cs_xnas_xnys_hybrid_2025-10-24.parquet"
df_hybrid = pl.read_parquet(hybrid_path)

print(f"Total universo híbrido: {len(df_hybrid):,}")

# Verificar si existe columna 'active'
if 'active' in df_hybrid.columns:
    hybrid_active = len(df_hybrid.filter(pl.col("active") == True))
    hybrid_inactive = len(df_hybrid) - hybrid_active
    print(f"  - Activos < $2B:  {hybrid_active:>6,} ({hybrid_active/len(df_hybrid)*100:>5.1f}%)")
    print(f"  - Inactivos ALL:  {hybrid_inactive:>6,} ({hybrid_inactive/len(df_hybrid)*100:>5.1f}%)")
else:
    print("  [Nota: columna 'active' no encontrada en el resultado final]")

# Distribución por exchange
if 'primary_exchange' in df_hybrid.columns:
    print()
    print("Distribución por exchange:")
    for exchange in ["XNAS", "XNYS"]:
        total = len(df_hybrid.filter(pl.col("primary_exchange") == exchange))
        active = len(df_hybrid.filter((pl.col("primary_exchange") == exchange) & (pl.col("active") == True)))
        inactive = total - active
        pct = total / len(df_hybrid) * 100
        print(f"  {exchange}: {total:>6,} ({pct:>5.1f}%) - {active:>5,} activos, {inactive:>5,} inactivos")

# =============================================================================
# PASO 5: Análisis de market_cap (solo en activos)
# =============================================================================
print("\n[PASO 5] ANÁLISIS DE MARKET CAP (SOLO ACTIVOS)")
print("-" * 100)

# Verificar cuántos activos tienen market_cap en el híbrido
if 'active' in df_hybrid.columns:
    activos_hybrid = df_hybrid.filter(pl.col('active') == True)
    
    # Hacer join con details para obtener market_cap
    df_with_details = activos_hybrid.join(
        df_details.select(['ticker', 'market_cap']),
        on='ticker',
        how='left'
    )
    
    with_cap = df_with_details.filter(pl.col('market_cap').is_not_null())
    without_cap = df_with_details.filter(pl.col('market_cap').is_null())
    
    print(f"Activos en híbrido:         {len(activos_hybrid):,}")
    print(f"  - Con market_cap:         {len(with_cap):,}")
    print(f"  - Sin market_cap:         {len(without_cap):,}")
    
    if len(with_cap) > 0:
        print()
        print("Estadísticas de market cap (activos):")
        stats = with_cap.select(pl.col("market_cap")).describe()
        display(stats)
        
        print()
        print("Rangos de market cap (activos):")
        ranges = [
            ("Nano cap (< $50M)", 0, 50_000_000),
            ("Micro cap ($50M - $300M)", 50_000_000, 300_000_000),
            ("Small cap ($300M - $2B)", 300_000_000, 2_000_000_000),
        ]
        
        for label, min_val, max_val in ranges:
            count = len(with_cap.filter(
                (pl.col("market_cap") >= min_val) & (pl.col("market_cap") < max_val)
            ))
            pct = count / len(with_cap) * 100
            print(f"  {label}: {count:>6,} ({pct:>5.1f}%)")
        
        # Verificar si hay alguno >= $2B (error en filtrado)
        over_2b = len(with_cap.filter(pl.col("market_cap") >= 2_000_000_000))
        if over_2b > 0:
            print(f"\n⚠️  ADVERTENCIA: {over_2b} activos con market_cap >= $2B (error en filtrado)")

print("\nNOTA: Los 5,594 inactivos NO tienen market_cap (Polygon no proporciona)")

# =============================================================================
# PASO 6: Muestreo de datos
# =============================================================================
print("\n[PASO 6] MUESTREO DE DATOS FINALES")
print("-" * 100)

print("\nPrimeros 10 tickers ACTIVOS (ordenados alfabéticamente):")
cols_display = ["ticker", "name", "primary_exchange", "active"]

if 'ticker' in df_hybrid.columns and 'active' in df_hybrid.columns:
    activos_sample = df_hybrid.filter(pl.col('active') == True).sort("ticker")
    display(activos_sample.select([col for col in cols_display if col in activos_sample.columns]).head(10))

print("\nPrimeros 10 tickers INACTIVOS (ordenados alfabéticamente):")
if 'ticker' in df_hybrid.columns and 'active' in df_hybrid.columns:
    inactivos_sample = df_hybrid.filter(pl.col('active') == False).sort("ticker")
    display(inactivos_sample.select([col for col in cols_display if col in inactivos_sample.columns]).head(10))

# =============================================================================
# PASO 7: Resumen ejecutivo
# =============================================================================
print("\n" + "=" * 100)
print("RESUMEN EJECUTIVO")
print("=" * 100)

print(f"""
PIPELINE COMPLETO:
  1. Snapshot original (todos tipos): {len(df_original):>6,} tickers
  2. Filtro CS:                       {len(df_original.filter(pl.col('type') == 'CS')):>6,} tickers
  3. Filtro CS + XNAS/XNYS:           {len(df_cs_filtered):>6,} tickers
  4. Details descargados:             {len(df_details):>6,} tickers
  5. Con market_cap data:             {len(df_details.filter(pl.col('market_cap').is_not_null())):>6,} tickers (solo activos)
  6. Universo HÍBRIDO final:          {len(df_hybrid):>6,} tickers ✅

COMPOSICIÓN DEL UNIVERSO HÍBRIDO:
  - Activos < $2B:    {hybrid_active:>6,} tickers ({hybrid_active/len(df_hybrid)*100:>5.1f}%) [filtrados por market_cap]
  - Inactivos ALL:    {hybrid_inactive:>6,} tickers ({hybrid_inactive/len(df_hybrid)*100:>5.1f}%) [TODOS incluidos]
  
  TOTAL:              {len(df_hybrid):>6,} tickers

DISTRIBUCIÓN:
  - NASDAQ (XNAS):    {len(df_hybrid.filter(pl.col('primary_exchange') == 'XNAS')):>6,} tickers
  - NYSE (XNYS):      {len(df_hybrid.filter(pl.col('primary_exchange') == 'XNYS')):>6,} tickers

CARACTERÍSTICAS:
  ✅ Common Stock (CS) solamente
  ✅ NASDAQ (XNAS) y NYSE (XNYS) únicamente
  ✅ Activos filtrados por market_cap < $2B
  ✅ Inactivos incluidos SIN FILTRAR (evita survivorship bias)
  ✅ Sin sesgo de supervivencia

FUNDAMENTO (López de Prado):
  "Survivorship bias is one of the most severe biases in backtesting"
  - Los inactivos son CRÍTICOS para entrenar modelos de pump & dump
  - Contienen las señales más fuertes de pumps terminales
  - Permiten feature engineering: days_to_delisting, is_terminal_pump

ARCHIVOS GENERADOS:
  ✅ {cs_filtered_path}
  ✅ {details_path}
  ✅ {hybrid_path}
  ✅ processed/universe/cs_xnas_xnys_hybrid_2025-10-24.csv
""")

print("=" * 100)
print("VERIFICACIÓN COMPLETADA")
print("=" * 100)